## Installing Libraries

In [1]:
# Uninstall existing libraries
!pip uninstall -y tensorflow transformers

# Reinstall TensorFlow and Transformers
!pip install tensorflow==2.11.0  # Ensure compatibility; adjust version as needed
!pip install transformers==4.33.0  # Ensure compatibility; adjust version as needed


Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 55.3 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tenso

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1


## Data Loading

In [1]:
from huggingface_hub import login
login()

In [2]:
!pip install datasets



In [3]:
from datasets import load_dataset

# Load the dataset and specify the split
ds = load_dataset("ServiceNow/repliqa", split='repliqa_0')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
ds.column_names

['document_id',
 'document_topic',
 'document_path',
 'document_extracted',
 'question_id',
 'question',
 'answer',
 'long_answer']

## Data Preprocessing

In [5]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-small')

def preprocess_function(examples):
    # Create the input text
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples['question'], examples['document_extracted'])]
    # Prepare the target answers
    targets = examples['answer']

    # Tokenize inputs and targets
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing
tokenized_ds = ds.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/17955 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Model Loading , Fine Tuning , Optimiztion

In [6]:
!pip install peft
!pip install transformers datasets accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 8.1 MB/s eta 0:00:00


In [8]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=40e52476e70fdd5102794063d0f724828b259904721547d358c1bc3d4ea7d55b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [9]:
import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig
from datasets import load_metric

# Load the dataset
dataset = load_dataset("ServiceNow/repliqa", split='repliqa_0')

# Split the dataset into training and evaluation sets
dataset = dataset.shuffle(seed=42)
split_dataset = dataset.train_test_split(test_size=0.1)

train_ds = split_dataset['train']
eval_ds = split_dataset['test']

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def preprocess_function(examples):
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples['question'], examples['document_extracted'])]
    targets = examples['answer']

    # Tokenize inputs and labels
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=targets, max_length=128, truncation=True, padding="max_length")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing to both train and eval datasets
tokenized_train_ds = train_ds.map(preprocess_function, batched=True)
tokenized_eval_ds = eval_ds.map(preprocess_function, batched=True)

# Load model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Now proceed with your training setup


# Set up LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Evaluation metric: ROUGE
rouge_metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Add mean to get the average score for each metric
    result = {key: value.mid.fmeasure for key, value in result.items()}

    return result

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='./results',
        num_train_epochs=2,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        warmup_steps=500,
        weight_decay=0.01,
        fp16=True,  # Use float16 precision
        logging_dir=None,  # This disables logging to a directory
        logging_steps=1000,  # Set this to a large number to effectively stop logging
        remove_unused_columns=False  # Prevent ignoring columns not used by the model

    ),
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_train_ds,
)
trainer.train()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/16159 [00:00<?, ? examples/s]

Map:   0%|          | 0/1796 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
1000,6.605100
2000,0.429500
3000,0.357600
4000,0.323100
5000,0.325300
6000,0.314400
7000,0.308900
8000,0.301000


TrainOutput(global_step=8080, training_loss=1.1127372191684082, metrics={'train_runtime': 888.5351, 'train_samples_per_second': 36.372, 'train_steps_per_second': 9.094, 'total_flos': 4403255467573248.0, 'train_loss': 1.1127372191684082, 'epoch': 2.0})

## Model Saving

In [10]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Define paths
model_path = '/content/drive/Starred/TensorGoModel'

# Save the model and tokenizer locally
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/Starred/TensorGoModel/tokenizer_config.json',
 '/content/drive/Starred/TensorGoModel/special_tokens_map.json',
 '/content/drive/Starred/TensorGoModel/spiece.model',
 '/content/drive/Starred/TensorGoModel/added_tokens.json')

## Model Evaluation

In [60]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from datasets import load_metric

# Load the tokenizer (should be the same tokenizer used during training)
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Load the fine-tuned model
# Replace this with your actual fine-tuned model loading method
fine_tuned_model = trainer.model

# Load the pre-trained model (for comparison)
pretrained_model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Ensure the device is set correctly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)
pretrained_model.to(device)

def generate_and_evaluate(question, context, reference_answer):
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate the output with the fine-tuned model
    fine_tuned_output_sequences = fine_tuned_model.generate(input_ids=inputs['input_ids'], max_length=50, num_beams=4, early_stopping=True)
    fine_tuned_generated_answer = tokenizer.decode(fine_tuned_output_sequences[0], skip_special_tokens=True)

    # Generate the output with the pre-trained model
    pretrained_output_sequences = pretrained_model.generate(input_ids=inputs['input_ids'], max_length=50, num_beams=4, early_stopping=True)
    pretrained_generated_answer = tokenizer.decode(pretrained_output_sequences[0], skip_special_tokens=True)

    # Compute ROUGE score
    rouge = load_metric('rouge')
    fine_tuned_scores = rouge.compute(predictions=[fine_tuned_generated_answer], references=[reference_answer])
    pretrained_scores = rouge.compute(predictions=[pretrained_generated_answer], references=[reference_answer])

    # Extract ROUGE F1 score
    fine_tuned_rouge_f1 = fine_tuned_scores['rougeL'].mid.fmeasure
    pretrained_rouge_f1 = pretrained_scores['rougeL'].mid.fmeasure

    # Print the generated answers
    print(f"Fine-Tuned Model Answer: {fine_tuned_generated_answer}")
    print(f"Pre-Trained Model Answer: {pretrained_generated_answer}")

    # Print ROUGE-L F1 scores
    print(f"\nFine-Tuned Model ROUGE-L F1 Score: {fine_tuned_rouge_f1}")
    print(f"Pre-Trained Model ROUGE-L F1 Score: {pretrained_rouge_f1}")

# Get user inputs
question = "What have architects and engineers advocated for in terms of reconstruction in San Juan post-Hurricane Elara?"
context = "Hurricane Elara has struck San Juan with unprecedented force, hitting the city on October 10 and causing extensive damage across the Caribbean island. With Elara reaching Category 5 status, winds exceeded 160 mph, leading to severe flooding, power outages, and widespread infrastructure damage. The death toll currently stands at 47, with many still missing. Recovery efforts are underway, but criticism of the government’s response has intensified, with survivors and political leaders questioning the adequacy of aid distribution and emergency preparedness. The storm has highlighted longstanding issues with infrastructure neglect and the broader impact of climate change, as scientists note that increasingly intense storms are becoming the new norm. As San Juan begins to rebuild, there is a call for innovative and resilient infrastructure, while economic uncertainty looms due to the cost of reconstruction and disruptions to tourism and trade. The long-term response will need to address disaster preparedness, climate change mitigation, and infrastructure reforms to ensure a more secure future for the city."
reference_answer = "Architects and engineers advocated for new and strong building projects that can stay safe during future storms"

# Generate outputs and evaluate
generate_and_evaluate(question, context, reference_answer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fine-Tuned Model Answer: The answer is not found in the document.
Pre-Trained Model Answer: innovative and resilient infrastructure

Fine-Tuned Model ROUGE-L F1 Score: 0.0
Pre-Trained Model ROUGE-L F1 Score: 0.09523809523809523


In [61]:
#What have architects and engineers advocated for in terms of reconstruction in San Juan post-Hurricane Elara?


#Hurricane Elara has struck San Juan with unprecedented force, hitting the city on October 10 and causing extensive damage across the Caribbean island. With Elara reaching Category 5 status, winds exceeded 160 mph, leading to severe flooding, power outages, and widespread infrastructure damage. The death toll currently stands at 47, with many still missing. Recovery efforts are underway, but criticism of the government’s response has intensified, with survivors and political leaders questioning the adequacy of aid distribution and emergency preparedness. The storm has highlighted longstanding issues with infrastructure neglect and the broader impact of climate change, as scientists note that increasingly intense storms are becoming the new norm. As San Juan begins to rebuild, there is a call for innovative and resilient infrastructure, while economic uncertainty looms due to the cost of reconstruction and disruptions to tourism and trade. The long-term response will need to address disaster preparedness, climate change mitigation, and infrastructure reforms to ensure a more secure future for the city.

#Architects and engineers advocated for new and strong building projects that can stay safe during future storms